In [ ]:
# Filter raw data from planting to heading date
loc_files = './soil' # <--- address to raw data here

wheat_df = pd.read_excel('sws_spring_wheat.xlsx',sheet_name='Sheet2')
for file_name in os.listdir(loc_files):
    if file_name.endswith('.xlsx'):
        # Load the datasets
        file_path = os.path.join(loc_files,file_name)
        loc_df = pd.read_excel(file_path)


        # Convert date columns to datetime format
        loc_df['date'] = pd.to_datetime(loc_df['date'])
        wheat_df['planting date'] = pd.to_datetime(wheat_df['planting date'])
        wheat_df['harvest date'] = pd.to_datetime(wheat_df['harvest date'])

        # Initialize an empty DataFrame to store the filtered results
        filtered_df = pd.DataFrame()
        location, extension = os.path.splitext(file_name)
        # Loop through each row of the growth data frame (sws_spring_wheat)
        for index, row in wheat_df.iterrows():
            if row['location'] == location:
                # Filter rows where the date is between the planting and harvest dates for the respective year
                mask = (loc_df['date'] >= row['planting date']) & (loc_df['date'] <= row['harvest date'])
                filtered_data = loc_df[mask]
                # Append the filtered data to the filtered_df
                filtered_df = pd.concat([filtered_df, filtered_data], ignore_index=True)
                
        output_file_path = os.path.join('./soil_p_h', f'{file_name}')
        filtered_df.to_excel(output_file_path, index=False)

In [ ]:
### Adding the new filters variable to the old data ###
file_path1 = './test' # <--- address to old data here
file_path2 = './pdsi_p_h' # <--- address to new data here
output_path = './p_h_with_strs' # <--- address to output data here
excel_files = [f for f in os.listdir(file_path1) if f.endswith('.xlsx')]

for excel_f in excel_files:
    df1 = pd.read_excel(os.path.join(file_path1, excel_f))
    df2 = pd.read_excel(os.path.join(file_path2, excel_f))
    df1['date'] = pd.to_datetime(df1['date'])
    df2['date'] = pd.to_datetime(df2['date'])
    # Mergge according to the data
    df1 = df1.merge(df2[['date', 'pdsi']], on = 'date', how = 'left')
    output_file = os.path.join(output_path, f'{os.path.splitext(excel_f)[0]}.xlsx')
    df1.to_excel(output_file, index = False)

In [ ]:
## Set dataframe. Taking average or sum of variables, considering which one makes sense, during their repsective growth stage ##

file_path = './test1/' # <--- address to prepared daily data here

# gs here stands for growth stage ##
output_file_path = './gs_data/' # <--- address to aggregated data (weekly, montly, seasonal or growth stage) here
# List all .xlsx files in the directory
excel_files = [f for f in os.listdir(file_path) if f.endswith('.xlsx')]

# Loop through each Excel file
for excel_f in excel_files:
    # Read the Excel file into a DataFrame
    df = pd.read_excel(os.path.join(file_path, excel_f))
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    gs_agg = df.groupby(['stage', 'year']).agg({
            'precip': 'sum',
            'tmax': 'mean',
            'tmin': 'mean',
            'Tavg': 'mean',
            'gdd': 'sum',
            'dgdd': 'sum',
            'dtr': 'mean',
            'prdtr': 'mean',
            'pet': 'sum',
            'etr': 'mean',
            'srad': 'mean',
            'rmax': 'mean',
            'rmin': 'mean',
            'vs': 'mean',
            'ravg': 'mean',
            'vpd': 'mean',
            'fdd':'sum',
            'hdd':'sum',
            'spei':'mean',
            'pdsi':'mean',
            'soil':'mean'}).reset_index()
    stage_order = {"Emergence": 0, "Tillering": 1, "Jointing": 2, "Heading": 3, "Flowering": 4, "Grain fill": 5, "Maturity": 6, "Beyond Maturity": 7}
    gs_agg["stage_order"] = gs_agg["stage"].map(stage_order)
    gs_agg = gs_agg.sort_values(by=["stage_order","year"]).drop(columns="stage_order")
    gs_agg = gs_agg.reset_index(drop=True)

    # Calculate the number of days in each stage grouped by 'year' and 'stage'
    days_per_stage_year = df.groupby(['year', 'stage'])['dap'].agg(lambda x: x.max() - x.min()).reset_index()
    days_per_stage_year.rename(columns={'dap': '#days'}, inplace=True)

    # Calculate the total number of wet days in each stage grouped by 'year' and 'stage'
    wet_per_stage_year = df.groupby(['year', 'stage'])['wet'].agg(sum).reset_index()
    wet_per_stage_year.rename(columns={'wet': '#wet'}, inplace=True)

    # Merge the calculated values back into the aggregated dataset
    gs_agg = gs_agg.merge(days_per_stage_year, on=['year', 'stage'], how='left')
    gs_agg = gs_agg.merge(wet_per_stage_year, on=['year', 'stage'], how='left')

    # Calculate wet frequency
    gs_agg['wet_frequency'] = gs_agg['#wet'] / gs_agg['#days']

    # Save the output to an Excel file
    output_file = os.path.join(output_file_path, f'{os.path.splitext(excel_f)[0]}.xlsx')
    gs_agg.to_excel(output_file, index=False)

In [ ]:
#Now data are in seperate files based on location names, So I need th combine them into one file
## Combine all data frames ##
directory = './gs_data'
# List to store dataframes
dataframes = []

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".xlsx"):  # check for Excel files
        filepath = os.path.join(directory, filename)
        
        # Read the Excel file
        df = pd.read_excel(filepath)
        
        location, extension = os.path.splitext(filename)

        # Add a column with the filename (without extension)
        df['location'] = location
        
        # Append dataframe to the list
        dataframes.append(df)

# Concatenate all the dataframes
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df = combined_df.drop(columns = ['#wet'])
combined_df.to_excel('./combined_gs.xlsx', index=False)

In [ ]:
### Rehape the data so it will have columns based on growth stage and evironmental variable ###

# Define the order of the stages
stage_order = ["Emergence", "Tillering", "mature", "Jointing", "Heading", "Flowering", "Grain fill", "Maturity", "Beyond Maturity"]  

# Convert 'stage' to a categorical type with the specified order
combined_df["stage"] = pd.Categorical(combined_df["stage"], categories=stage_order, ordered=True)

# Pivot the data
reshaped_df = combined_df.pivot(index=["year", "location"], columns="stage")

# Flatten the multi-level column index
reshaped_df.columns = [f"{col[1]}_{col[0]}" for col in reshaped_df.columns]

# Reset the index for a clean format
reshaped_df.reset_index(inplace=True)
# Save the reshaped DataFrame to an Excel file
reshaped_df.to_excel('./reshaped_combined_gs.xlsx', index=False)

In [ ]:
#################################################################################################
## I want to add precipitation (or any other variable) before each growing season to the data  ##
## I will sum up the precipitation from october until planting date to the data                ##
#################################################################################################

from datetime import datetime

loc_files = './spei' # <---- address to raw data here
output_path = './spei_season' # <--- address to output added preseason data here
wheat_df = pd.read_excel('sws_spring_wheat.xlsx',sheet_name='Sheet2')
yearly_precip_sums = {}
for file_name in os.listdir(loc_files):
    if file_name.endswith('.xlsx'):
        # Load the datasets
        file_path = os.path.join(loc_files,file_name)
        loc_df = pd.read_excel(file_path)


        # Convert date columns to datetime format
        loc_df['date'] = pd.to_datetime(loc_df['date'])
        wheat_df['planting date'] = pd.to_datetime(wheat_df['planting date'])
        #wheat_df['harvest date'] = pd.to_datetime(wheat_df['harvest date'])

        
        # Initialize an empty DataFrame to store the filtered results
        filtered_df = pd.DataFrame()
        location, extension = os.path.splitext(file_name)
        # Loop through each row of the growth data frame (sws_spring_wheat)
        for index, row in wheat_df.iterrows():
            if row['location'] == location:
                year = row['planting date'].year - 1
                start_date = datetime.strptime(f'10/1/{year}', '%m/%d/%Y')
                
                # Apply the date filtering
                mask = (loc_df['date'] >= start_date) & (loc_df['date'] < row['planting date'])
                filtered_data = loc_df[mask]
               # Calculate the total precipitation for the year
                yearly_pre_var_sums[year +1] = filtered_data['spei'].sum()
        
        # Convert the dictionary to a DataFrame
        pre_var_df = pd.DataFrame(list(yearly_pre_var_sums.items()), columns=['year', 'preseason_spei'])
        
        # Save the DataFrame to the output directory
        output_file_path = os.path.join(output_path, f'{location}.xlsx')
        pre_var_df.to_excel(output_file_path, index=False)

In [ ]:
# I want to add the preseason to the reshaped data
directory = './pr_season'
# List to store dataframes
dataframes = []

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".xlsx") :  # check for Excel files
        filepath = os.path.join(directory, filename)
        
        # Read the Excel file
        df = pd.read_excel(filepath)
        
        location, extension = os.path.splitext(filename)

        # Add a column with the filename (without extension)
        df['location'] = location
        
        # Append dataframe to the list
        dataframes.append(df)

# Concatenate all the dataframes
combined_df = pd.concat(dataframes, ignore_index=True)
# read the reshaped data
reshaped_df = pd.read_excel('./reshaped_combined_gs.xlsx')

reshaped_df = reshaped_df.merge(combined_df[['year', 'location', 'preseason_precip']], on=['year', 'location'], how='left')
# save the reshaped data
reshaped_df.to_excel('./reshaped_combined_gs_with_preseason.xlsx', index=False)

In [ ]:
#############################################
#### I want to merge soil static data    ####
#############################################
import pandas as pd
df_soil = pd.read_excel('./soil_data/locations.xlsx')
#df_soil = df_soil.drop(columns = ['Latitude', 'Longitude'])

# read environment file and merge it with crop trait
df_env = pd.read_csv('f_data_gs2.csv')
df_merged = df_soil.merge(df_env, on = ['location'], how = 'left')
df_merged.to_csv('./df_22805.csv')